# Forward Kinematics (FK) Fundamentals — Interactive Notebook

This notebook is designed for a **university-level introduction to robotics** course and is intended to run on **Binder**.
It introduces forward kinematics step-by-step using interactive visualizations:

1. **1-DOF planar arm** (single revolute joint)
2. **2-DOF planar arm** (two revolute joints)
3. Building FK using **rotation matrices** and **homogeneous transforms**
4. Verifying results with **symbolic math (SymPy)**

**Learning goals**
- Derive FK for simple manipulators
- Understand composition of transforms (frame-to-frame multiplication)
- Connect geometry formulas to matrix-based robotics notation
- Visualize how joint angles map to end-effector position


## Binder notes (read me)

This notebook uses:
- `numpy`, `matplotlib`
- `ipywidgets` for interactivity
- `sympy` for symbolic derivations

Binder images for JupyterLab typically support `ipywidgets` out of the box.
If widgets do not render, run the setup cell below (it is safe on Binder, but may take a minute).


In [11]:
# If widgets don't appear, uncomment the next lines (Binder / fresh environments):
# %pip -q install ipywidgets sympy matplotlib numpy
# %pip -q install jupyterlab_widgets

import numpy as np
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display, Markdown


## Helper functions

We'll work in the **plane** (2D) and use:
- 2×2 rotation matrices `R(θ)`
- 3×3 homogeneous transforms `T(θ, x, y)` for planar motion

Homogeneous transforms let us compose rotations + translations with matrix multiplication.


In [13]:
def rot2(theta):
    # 2D rotation matrix
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s],
                     [s,  c]])

def T2(theta, tx, ty):
    # 2D homogeneous transform (3x3): rotation + translation
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s, tx],
                     [s,  c, ty],
                     [0,  0,  1]])

def apply_T(T, p):
    # Apply 3x3 transform T to 2D point p=(x,y)
    ph = np.array([p[0], p[1], 1.0])
    qh = T @ ph
    return qh[:2]

def setup_axes(ax, L):
    ax.set_aspect('equal', adjustable='box')
    ax.grid(True, alpha=0.3)
    ax.set_xlim([-L, L])
    ax.set_ylim([-L, L])
    ax.set_xlabel("x")
    ax.set_ylabel("y")

def draw_frame(ax, origin, x_axis, y_axis, scale=0.3, label=None):
    # Draw a small coordinate frame at origin with given axis directions (unit vectors)
    ox, oy = origin
    ax.arrow(ox, oy, scale*x_axis[0], scale*x_axis[1],
             head_width=0.05, length_includes_head=True)
    ax.arrow(ox, oy, scale*y_axis[0], scale*y_axis[1],
             head_width=0.05, length_includes_head=True)
    if label:
        ax.text(ox + 0.05, oy + 0.05, label)

def plot_planar_arm(points, L=2.5, show_frames=False, frame_orientations=None, title=""):
    '''
    points: list of joint positions in world frame: [p0, p1, ..., pn] with p0=(0,0).
    frame_orientations: list of angles for frames at each joint (len = len(points)), used if show_frames.
    '''
    fig, ax = plt.subplots(figsize=(6, 6))
    setup_axes(ax, L)
    xs = [p[0] for p in points]
    ys = [p[1] for p in points]

    ax.plot(xs, ys, marker='o')
    ax.scatter(xs[-1], ys[-1], s=80)  # end-effector

    if show_frames and frame_orientations is not None:
        for i, (p, ang) in enumerate(zip(points, frame_orientations)):
            R = rot2(ang)
            draw_frame(ax, p, R[:, 0], R[:, 1], scale=0.35, label=f"{{{i}}}")

    ax.set_title(title)
    return fig, ax


\
# 1) One revolute joint (1-DOF planar arm)

Consider a single link of length \(L_1\) attached at the origin with a **revolute** joint angle \(\theta_1\).

**Geometry approach** (from a right triangle):

\[
x = L_1 \cos\theta_1,\qquad y = L_1 \sin\theta_1.
\]

**Transform approach** (homogeneous transform):

A rotation by \(\theta_1\) and then a translation of \(L_1\) along the **local x-axis** is:

\(
{}^{0}\!T_{1} = \text{Rot}(\theta_1)\, \text{Trans}(L_1, 0)
\)

Below: interactively explore how \(\theta_1\) and \(L_1\) change the end-effector position.


In [17]:
def fk_1dof(theta1, L1):
    # End-effector position for a single link
    return np.array([L1*np.cos(theta1), L1*np.sin(theta1)])

def show_1dof(theta1_deg=30.0, L1=1.0, show_frames=True):
    theta1 = np.deg2rad(theta1_deg)
    p0 = np.array([0.0, 0.0])
    p1 = fk_1dof(theta1, L1)
    points = [p0, p1]

    plot_planar_arm(
        points,
        L=max(2.5, L1+0.8),
        show_frames=show_frames,
        frame_orientations=[0.0, theta1],
        title="1-DOF Planar Arm"
    )

    plt.text(-0.95*max(2.5, L1+0.8), 0.85*max(2.5, L1+0.8),
             f"θ₁ = {theta1_deg:.1f}°\nL₁ = {L1:.2f}\nEnd-effector: (x, y)=({p1[0]:.3f}, {p1[1]:.3f})",
             fontsize=11, bbox=dict(boxstyle="round", alpha=0.15))

    plt.show()

theta1_slider = widgets.FloatSlider(value=35.0, min=-180, max=180, step=1, description='θ₁ (deg)')
L1_slider = widgets.FloatSlider(value=1.0, min=0.2, max=2.5, step=0.05, description='L₁')
frames_toggle = widgets.Checkbox(value=True, description='Show frames')

ui = widgets.VBox([theta1_slider, L1_slider, frames_toggle])
out = widgets.interactive_output(show_1dof, {
    "theta1_deg": theta1_slider,
    "L1": L1_slider,
    "show_frames": frames_toggle
})

display(ui, out)


Output()

\
### Quick check (1-DOF)

1. Set \(L_1=2\). What is the end-effector position at \(\theta_1=0^\circ\)? at \(90^\circ\)?  
2. At what angles is \(x=0\)? At what angles is \(y=0\)?  
3. Why does increasing \(L_1\) scale both the x and y coordinates?


\
# 2) Two revolute joints (2-DOF planar arm)

Now consider two links of lengths \(L_1, L_2\) with joint angles \(\theta_1, \theta_2\).  
Angle \(\theta_2\) is measured **relative to link 1** (standard serial-chain convention).

## Geometry derivation

The elbow position is:
\(
p_1 = \begin{bmatrix} L_1\cos\theta_1 \\ L_1\sin\theta_1 \end{bmatrix}
\)

The second link is rotated by \(\theta_1+\theta_2\) in the world frame, so the end-effector is:
\(
p_2 = p_1 + \begin{bmatrix} L_2\cos(\theta_1+\theta_2) \\ L_2\sin(\theta_1+\theta_2) \end{bmatrix}
\)

Therefore:
\(
x = L_1\cos\theta_1 + L_2\cos(\theta_1+\theta_2),\qquad
y = L_1\sin\theta_1 + L_2\sin(\theta_1+\theta_2)
\)

## Transform derivation (homogeneous transforms)

Define:
- \(^{0}\!T_{1} = \text{Rot}(\theta_1)\,\text{Trans}(L_1, 0)\)
- \(^{1}\!T_{2} = \text{Rot}(\theta_2)\,\text{Trans}(L_2, 0)\)

Then:
\(
^{0}\!T_{2} = {}^{0}\!T_{1}\,{}^{1}\!T_{2}
\)

Below: interactively explore 2-DOF FK and compare the geometry formula to the matrix product.


In [ ]:
def fk_2dof(theta1, theta2, L1, L2):
    # Geometry formula
    x = L1*np.cos(theta1) + L2*np.cos(theta1 + theta2)
    y = L1*np.sin(theta1) + L2*np.sin(theta1 + theta2)
    return np.array([x, y])

def fk_2dof_transforms(theta1, theta2, L1, L2):
    # Transform approach: Rot(theta) then translate along local x
    T01 = T2(theta1, 0, 0) @ T2(0, L1, 0)
    T12 = T2(theta2, 0, 0) @ T2(0, L2, 0)
    T02 = T01 @ T12
    p = apply_T(T02, (0, 0))
    return p, T01, T12, T02

def show_2dof(theta1_deg=30.0, theta2_deg=45.0, L1=1.0, L2=1.0, show_frames=True):
    th1 = np.deg2rad(theta1_deg)
    th2 = np.deg2rad(theta2_deg)

    p0 = np.array([0.0, 0.0])
    p1 = np.array([L1*np.cos(th1), L1*np.sin(th1)])
    p2_geo = fk_2dof(th1, th2, L1, L2)
    p2_T, T01, T12, T02 = fk_2dof_transforms(th1, th2, L1, L2)

    points = [p0, p1, p2_geo]
    plot_planar_arm(
        points,
        L=max(2.5, L1+L2+0.8),
        show_frames=show_frames,
        frame_orientations=[0.0, th1, th1+th2],
        title="2-DOF Planar Arm"
    )

    err = np.linalg.norm(p2_geo - p2_T)

    plt.text(-0.95*max(2.5, L1+L2+0.8), 0.85*max(2.5, L1+L2+0.8),
             f"θ₁ = {theta1_deg:.1f}°, θ₂ = {theta2_deg:.1f}°\n"
             f"L₁ = {L1:.2f}, L₂ = {L2:.2f}\n"
             f"End-effector (geometry): ({p2_geo[0]:.3f}, {p2_geo[1]:.3f})\n"
             f"End-effector (transforms): ({p2_T[0]:.3f}, {p2_T[1]:.3f})\n"
             f"Difference norm: {err:.2e}",
             fontsize=10, bbox=dict(boxstyle="round", alpha=0.15))

    plt.show()

theta1_slider = widgets.FloatSlider(value=30.0, min=-180, max=180, step=1, description='θ₁ (deg)')
theta2_slider = widgets.FloatSlider(value=45.0, min=-180, max=180, step=1, description='θ₂ (deg)')
L1_slider = widgets.FloatSlider(value=1.0, min=0.2, max=2.5, step=0.05, description='L₁')
L2_slider = widgets.FloatSlider(value=1.0, min=0.2, max=2.5, step=0.05, description='L₂')
frames_toggle = widgets.Checkbox(value=True, description='Show frames')

ui = widgets.VBox([theta1_slider, theta2_slider, widgets.HBox([L1_slider, L2_slider]), frames_toggle])
out = widgets.interactive_output(show_2dof, {
    "theta1_deg": theta1_slider,
    "theta2_deg": theta2_slider,
    "L1": L1_slider,
    "L2": L2_slider,
    "show_frames": frames_toggle
})

display(ui, out)


\
## 2-DOF FK via symbolic matrix multiplication (SymPy)

In robotics, it’s common to derive FK by multiplying transforms and then simplifying the result.

In 2D, we used:
\(
^{0}\!T_{1} = \text{Rot}(\theta_1)\text{Trans}(L_1,0),\quad
^{1}\!T_{2} = \text{Rot}(\theta_2)\text{Trans}(L_2,0)
\)

Let’s compute \(^{0}\!T_{2}\) symbolically and inspect the translation terms \(x(\theta_1,\theta_2)\), \(y(\theta_1,\theta_2)\).


In [ ]:
import sympy as sp

theta1, theta2, L1, L2 = sp.symbols('theta1 theta2 L1 L2', real=True)

def T2_sym(theta, tx, ty):
    c, s = sp.cos(theta), sp.sin(theta)
    return sp.Matrix([[c, -s, tx],
                      [s,  c, ty],
                      [0,  0,  1]])

T01 = T2_sym(theta1, 0, 0) * T2_sym(0, L1, 0)
T12 = T2_sym(theta2, 0, 0) * T2_sym(0, L2, 0)
T02 = sp.simplify(T01 * T12)

T02


\
Extract the end-effector position from the translation part of \(^{0}\!T_2\):

\(
x = (^{0}\!T_{2})_{0,2},\qquad y = (^{0}\!T_{2})_{1,2}
\)


In [ ]:
x_expr = sp.simplify(T02[0, 2])
y_expr = sp.simplify(T02[1, 2])
x_expr, y_expr


\
You should see exactly:

\(
x = L_1\cos\theta_1 + L_2\cos(\theta_1+\theta_2),\quad
y = L_1\sin\theta_1 + L_2\sin(\theta_1+\theta_2)
\)

This is a great sanity check: **geometry** and **transform multiplication** agree.


\
# Practice problems

### A) Derive FK yourself (2-DOF)

1. Start from:
   \(
   ^0T_1 = \text{Rot}(\theta_1)\text{Trans}(L_1,0),\quad
   ^1T_2 = \text{Rot}(\theta_2)\text{Trans}(L_2,0)
   \)
2. Multiply \(^{0}\!T_2 = ^0T_1\,^1T_2\).
3. Extract the translation terms \((x,y)\).

### B) What if \(\theta_2\) were measured from the world frame?

If the second link angle were \(\theta_2\) from the **world x-axis** instead of relative to link 1, the geometry would change.
Rewrite the FK equations for that alternative convention and test it by editing the code.

### C) Add a prismatic joint

Modify the 1-DOF or 2-DOF example so that one joint is **prismatic** (a sliding joint).
For example, keep \(\theta_1\) revolute but let \(L_1\) be controlled by a slider representing extension \(d\).


\
# Where to go next

- Extend to **3-DOF planar** arms (add \(\theta_3\), \(L_3\))
- Move to **3D** with 4×4 transforms, including rotations about x/y/z
- Introduce **Denavit–Hartenberg (DH) parameters** to standardize transform construction
- Use FK as the basis for Jacobians, velocity kinematics, and inverse kinematics
